# Read in Configuration And Adjust Where Files are Stored
First we will look the default configuration

Then we adjust the 'ssj_cdf_root_dir' parameter (which determines where the code searches for 'raw' data it needs)

In [1]:
from ssjlatbin.io import read_config
config = read_config('../scripts/default_config.toml')

In [2]:
#Show the configurable parameters
import pprint
pprint.pprint(config)

{'all_channels': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 'calculation': {'uncertainty_tolerance': 100},
 'dataframevar_to_cdfvar': {'glats': 'SC_GEOCENTRIC_LAT',
                            'glons': 'SC_GEOCENTRIC_LON',
                            'mlats': 'SC_AACGM_LAT',
                            'mlts': 'SC_AACGM_LTIME'},
 'hard_channels': [0, 1, 2, 3, 4, 5, 6, 7, 8],
 'io': {'parquet_root_dir': '/home/ec2-user/SageMaker/efs/data/ssj_latbin/parquet',
        'ssj_cdf_root_dir': '/home/ec2-user/SageMaker/efs/data/dmspssjdata/',
        'ssj_cdf_version': '1.1.4'},
 'latbin': {'delta_lat': 2, 'max_lat': 80},
 'soft_channels': [10, 11, 12, 13, 14, 15, 16, 17]}


In [11]:
#Make changes to the configuration so that data is stored with this notebook, and the correct version of the data is used
import os
config['io']['ssj_cdf_root_dir']='../dat'
config['io']['ssj_cdf_version']='1.1.5'
if not os.path.exists(config['io']['ssj_cdf_root_dir']):
    os.makedirs(config['io']['ssj_cdf_root_dir'])

# Download a few days of DMSP SSJ data from NASA

In [12]:
#Here we set the satellite we are going to use and the range of time we will examine
import datetime
dmsp_number = 13
sdt = datetime.datetime(2006,12,2)
edt = datetime.datetime(2006,12,20)

In [8]:
#This code downloaded the data from NASA Space Physics Data Facility servers
import requests
base_url = 'https://cdaweb.gsfc.nasa.gov/pub/data/dmsp/dmspf13/ssj/precipitating-electrons-ions'
def download_f13_cdf(dt):
    fn = f'dmsp-f13_ssj_precipitating-electrons-ions_{dt.year}{dt.month:02d}{dt.day:02d}_v1.1.5.cdf'
    url = base_url
    url += f'/{dt.year}/'
    url += fn
    response = requests.get(url)
    if response.status_code == 404:
        raise RuntimeError(f'URL {url} returned 404 page not found')
    else:
        localfn = os.path.join(config['io']['ssj_cdf_root_dir'],fn)
        with open(localfn,'wb') as f:
            f.write(response.content)
        print(f'Downloaded \n {url} \n to \n {localfn}')

dt = sdt
while dt <= edt:
    download_f13_cdf(dt)
    dt+=datetime.timedelta(days=1)

Downloaded 
 https://cdaweb.gsfc.nasa.gov/pub/data/dmsp/dmspf13/ssj/precipitating-electrons-ions/2006/dmsp-f13_ssj_precipitating-electrons-ions_20061202_v1.1.5.cdf 
 to 
 ../dat/dmsp-f13_ssj_precipitating-electrons-ions_20061202_v1.1.5.cdf
Downloaded 
 https://cdaweb.gsfc.nasa.gov/pub/data/dmsp/dmspf13/ssj/precipitating-electrons-ions/2006/dmsp-f13_ssj_precipitating-electrons-ions_20061203_v1.1.5.cdf 
 to 
 ../dat/dmsp-f13_ssj_precipitating-electrons-ions_20061203_v1.1.5.cdf
Downloaded 
 https://cdaweb.gsfc.nasa.gov/pub/data/dmsp/dmspf13/ssj/precipitating-electrons-ions/2006/dmsp-f13_ssj_precipitating-electrons-ions_20061204_v1.1.5.cdf 
 to 
 ../dat/dmsp-f13_ssj_precipitating-electrons-ions_20061204_v1.1.5.cdf
Downloaded 
 https://cdaweb.gsfc.nasa.gov/pub/data/dmsp/dmspf13/ssj/precipitating-electrons-ions/2006/dmsp-f13_ssj_precipitating-electrons-ions_20061205_v1.1.5.cdf 
 to 
 ../dat/dmsp-f13_ssj_precipitating-electrons-ions_20061205_v1.1.5.cdf
Downloaded 
 https://cdaweb.gsfc.nasa.go

# Now we load the SSJ raw data into a Pandas DataFrame

First we will set a few more configuration options here:
* 'delta_lat' sets the width of the bins (in degrees) into which each orbit is divided
* 'uncertainty_tolerance' has to do with filtering out data for which the 'signal-to-noise' is not strong (high uncertainty measurements)

The `get_orbit_numberd_ssj_range_dataframe`


In [13]:
from ssjlatbin.cdf import get_orbit_numbered_ssj_range_dataframe
config['calculation']['uncertainty_tolerance']=75
df = get_orbit_numbered_ssj_range_dataframe(dmsp_number,sdt,edt,config)

File not found for date 2006-12-01 00:00:00


/home/liamk/mirror/Projects/ssj_latbin/ssjlatbin/fluxcalculations.py:100: RuntimeWarning: invalid value encountered in true_divide
  too_uncert = integral_flux_std/integral_flux*100>=uncertainty_tolerance


In [14]:
df

,glats,glons,mlats,mlts,ele_soft_energy,ele_soft_number,ele_hard_energy,ele_hard_number,ele_total_energy,ele_total_number,...,ion_soft_number,ion_hard_energy,ion_hard_number,ion_total_energy,ion_total_number,time,solar_zenith_angle,orbit_number,orbit_start_time,dglats
2006-12-02 00:05:03,0.025382,276.892322,22.998147,18.409124,0.000561,7.975644e+10,0.0,0.0,0.000561,7.975644e+10,...,0.0,0.0,0.0,0.0,0.0,2006-12-02 00:05:03,100.075425,0.0,2006-12-02 00:05:03,0.058323
2006-12-02 00:05:04,0.083705,276.879132,23.024838,18.408437,0.000187,2.658548e+10,0.0,0.0,0.000187,2.658548e+10,...,0.0,0.0,0.0,0.0,0.0,2006-12-02 00:05:04,100.089155,0.0,2006-12-02 00:05:03,0.058323
2006-12-02 00:05:05,0.142029,276.865942,23.051620,18.407750,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,2006-12-02 00:05:05,100.102876,0.0,2006-12-02 00:05:03,0.058324
2006-12-02 00:05:06,0.200352,276.852752,23.078508,18.407063,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,2006-12-02 00:05:06,100.116586,0.0,2006-12-02 00:05:03,0.058326
2006-12-02 00:05:07,0.258679,276.839562,23.105492,18.406377,0.000936,1.329274e+11,0.0,0.0,0.000936,1.329274e+11,...,0.0,0.0,0.0,0.0,0.0,2006-12-02 00:05:07,100.130286,0.0,2006-12-02 00:05:03,0.058324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-12-19 23:09:16,-0.288074,290.868059,22.662584,18.303567,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,2006-12-19 23:09:16,98.015107,253.0,2006-12-19 21:27:32,0.058296
2006-12-19 23:09:17,-0.229778,290.854875,22.688552,18.303401,0.000374,5.317096e+10,0.0,0.0,0.000374,5.317096e+10,...,0.0,0.0,0.0,0.0,0.0,2006-12-19 23:09:17,98.030288,253.0,2006-12-19 21:27:32,0.058296
2006-12-19 23:09:18,-0.171482,290.841691,22.714618,18.303234,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,2006-12-19 23:09:18,98.045460,253.0,2006-12-19 21:27:32,0.058296
2006-12-19 23:09:19,-0.113186,290.828507,22.740797,18.303066,0.000000,0.000000e+00,0.0,0.0,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,2006-12-19 23:09:19,98.060625,253.0,2006-12-19 21:27:32,0.058296


In [ ]:
import pandas as pd

In [ ]:
from ssjlatbin.latbin_pandas import bin_by_latitude
config['latbin']['delta_lat']=3

In [ ]:
lbdf = bin_by_latitude(df,config)

In [ ]:
lbdf

In [ ]:
from ssjlatbin.io import dataframe_to_latbinned_flux

In [ ]:
t,lats,flag,fluxes = dataframe_to_latbinned_flux(lbdf,'ion_soft_number')

In [ ]:
from ssjlatbin.fluxcalculations import average_particle_energy
t,lats,_,influxs = dataframe_to_latbinned_flux(lbdf,'ion_soft_number')
t,lats,_,eefluxh = dataframe_to_latbinned_flux(lbdf,'ele_hard_energy')
t,lats,_,eeflux = dataframe_to_latbinned_flux(lbdf,'ele_total_energy')
t,lats,_,enflux = dataframe_to_latbinned_flux(lbdf,'ele_total_number')
t,lats,_,sza = dataframe_to_latbinned_flux(lbdf,'solar_zenith_angle')

eavg = average_particle_energy(eeflux,enflux)
eavg[enflux==0]=0 #Prevent NaN values if zero number flux

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

def latlabel(value,tick_number):
    ilat = int(np.round(value))
    if ilat>=len(lats):
        ilat=len(lats)-1
    return '{}'.format(lats[ilat])

N = lats > 0

f = plt.figure(figsize=(8,6),dpi=150)
ax = f.add_subplot(111)
mappable = ax.pcolormesh(t,np.arange(lats.size),influxs.T,vmax=1e12,norm=LogNorm(),cmap='Reds')
ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f.colorbar(mappable,label='Soft Ion Number Flux \n [particles/m^2]')
f.autofmt_xdate()

# f = plt.figure(figsize=(8,6),dpi=150)
# ax = f.add_subplot(111)
# mappable = ax.pcolormesh(t,np.arange(lats.size),enflux.T,vmax=1e13,norm=LogNorm())
# ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
# f.colorbar(mappable,label='Total Electron Number Flux \n [particles/m^2]')
# f.autofmt_xdate()

# f = plt.figure(figsize=(8,6),dpi=150)
# ax = f.add_subplot(111)
# mappable = ax.pcolormesh(t,np.arange(lats.size),eeflux.T,vmin=.0001,vmax=10,norm=LogNorm())
# ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
# f.colorbar(mappable,label='Total Electron Energy Flux \n [mW/m^2]')
# f.autofmt_xdate()

f3 = plt.figure(figsize=(8,6),dpi=150)
ax3 = f3.add_subplot(111)
mappable2 = ax3.pcolormesh(t,np.arange(lats.size),influxs.T,vmax=1e12,norm=LogNorm(),cmap='Reds')
mappable = ax3.pcolormesh(t,np.arange(lats.size),eefluxh.T,vmin=.01,vmax=10,norm=LogNorm())
ax3.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f3.colorbar(mappable,label='Hard Electron Energy Flux \n [mW/m^2]')
f3.autofmt_xdate()

f4 = plt.figure(figsize=(8,6),dpi=150)
ax4 = f4.add_subplot(111)
mappable = ax4.pcolormesh(t,np.arange(lats.size),eavg.T,cmap='plasma',vmin=100,vmax=10000)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f4.colorbar(mappable,label='Electron Average Energy \n [eV]')
f4.autofmt_xdate()

f5 = plt.figure(figsize=(8,6),dpi=150)
ax5 = f5.add_subplot(111)
mappable = ax5.pcolormesh(t,np.arange(lats.size),np.degrees(sza.T),cmap='plasma_r')
ax5.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f5.colorbar(mappable,label='Solar Zenith Angle [deg]')
f5.autofmt_xdate()

# for thisax in [ax,ax3,ax4]:
#     thisax.set_xlim([datetime.datetime(2000,7,14),datetime.datetime(2000,7,19)])